<a href="https://colab.research.google.com/github/marcellinusc/solar-radiation/blob/data-prep/ml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Initialization

In [0]:
#@title Run on TensorFlow 2.x

%tensorflow_version 2.x
from __future__ import absolute_import, division
from __future__ import print_function, unicode_literals

In [0]:
#@title Import relevant modules

import re
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from datetime import datetime
from tensorflow.keras import layers
from matplotlib import pyplot as plt

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 20
pd.options.display.float_format = "{:.1f}".format

print('Modules Imported')

##Data Preparation

In [0]:
#@title Load raw dataset
data = pd.read_csv('https://raw.githubusercontent.com/marcellinusc/solar-radiation/data-prep/datasets.csv')
print('Length of raw dataset:', str(len(data)))

In [0]:
#@title Check for null values & show data types

print('NULL DATA')
print('===================')
print(data.isnull().sum())
print('\nDATA TYPES')
print('==========================================')
print(data.info())

In [0]:
#@title Exclude raw data between sunset-sunrise

daylight = [data['Time'].values[x] > data['Sunrise'].values[x]
            and data['Time'].values[x] < data['Sunset'].values[x]
            for x in range(len(data))]
data['Daylight'] = daylight
data['Daylight'] = data['Daylight'].astype('float')

# Clean raw dataset from daylight value of 0
data = data[data.Daylight != 0]

# Copy clean dataset as original back up
data_backup = data.copy()

print('Length of clean datset:', str(len(data)))

In [0]:
#@title Merge columns 'Date' and 'Time'

x = [i.replace('12:00:00 AM', '') for i in data['Date']]

data['DateTime'] = x + data['Time']
data['DateTime'] = pd.to_datetime(data['DateTime'])

print('DateTime column added to dataframe')

In [0]:
#@title Convert UNIX time format to UTC

data["TimeConversion"] = pd.to_datetime(data["Time"], format="%H:%M:%S")

# Get the month of the year and the day of the month
data["Month"] = pd.to_datetime(data["UNIXTime"].astype(int),
                               unit="s").dt.month
data["Day"] = pd.to_datetime(data["UNIXTime"].astype(int),
                             unit="s").dt.day 

# Get the hour and the minute of the day
data["Hour"] = pd.to_datetime(data["TimeConversion"],
                              format="%H:%M:%S").dt.hour
#data["Minute"] = pd.to_datetime(data["TimeConversion"],
#                              format="%H:%M:%S").dt.minute

print('UNIX converted to UTC time format')

In [0]:
#@title Plot 'Radiation' from September 2016 to January 2017

data_viz = data.sort_values('DateTime', ascending=True)
data_viz.head(15)

plt.figure(figsize = (40,25))
plt.rcParams['agg.path.chunksize'] = 10000
plt.plot(data_viz['DateTime'], data_viz['Radiation'])

plt.xlabel('Time', fontsize = 30)
plt.xticks(fontsize = 25)
plt.ylabel('Radiation', fontsize = 30)
plt.yticks(fontsize = 25)

In [0]:
#@title Normalize values to Z-score

# Calculate the Z-scores of each column in the dataset:
data_mean = data.select_dtypes(include=['float64', 'int64']).mean()
data_std = data.select_dtypes(include=['float64', 'int64']).std()
data_norm = (data.select_dtypes(include=['float64', 'int64']) 
             - data_mean) / data_std

data['Radiation'] = data_norm['Radiation']
data['Temperature'] = data_norm['Temperature']
data['Pressure'] = data_norm['Pressure']
data['Radiation'] = data_norm['Radiation']
data['Humidity'] = data_norm['Humidity']
data['WindDirection'] = data_norm['WindDirection']
data['WindSpeed'] = data_norm['WindSpeed']
data['Month'] = data_norm['Month']
data['Day'] = data_norm['Day']
data['Hour'] = data_norm['Hour']

print("Values normalized")

In [0]:
#@title Show correlation between parameters

data_par = data[['Radiation', 'Temperature', 'Pressure', 'Humidity',
                  'WindDirection', 'WindSpeed', 'Month', 'Day', 'Hour'
                  ]].copy()

plt.figure(figsize = (10,10))
mask = np.tril(np.ones_like(data_par.corr(), dtype=np.bool))
sns.heatmap(data_par.corr(), mask=mask, cmap="YlGnBu", annot = True, fmt = '.3f')

In [0]:
#@title Show density histogram for each parameter
(
    fig,
    (
        (ax1, ax2, ax3),
        (ax4, ax5, ax6),
        (ax7, ax8, ax9),
    ),
) = plt.subplots(3, 3, figsize=(20, 20))

sns.distplot(data['Radiation'], ax=ax1)
sns.distplot(data['Temperature'], ax=ax2)
sns.distplot(data['Pressure'], ax=ax3)
sns.distplot(data['Humidity'], ax=ax4)
sns.distplot(data['WindDirection'], ax=ax5)
sns.distplot(data['WindSpeed'], ax=ax6)
sns.distplot(data['Month'], ax=ax7)
sns.distplot(data['Hour'], ax=ax8)
sns.distplot(data['Day'], ax=ax9)

In [0]:
#@title Show barplot between label and other parameter
(
    fig,
    (
        (ax1, ax2),
        (ax3, ax4),
        (ax5, ax6),
        (ax7, ax8),
    ),
) = plt.subplots(4, 2, figsize=(20, 20))

ax1.set_title("Radiation vs Temperature")
sns.barplot(x="Temperature", y="Radiation",
            data=data, palette="GnBu_d", ax=ax1)
ax2.set_title("Radiation vs Pressure")
sns.barplot(x="Pressure", y="Radiation",
            data=data, palette="GnBu_d", ax=ax2)
ax3.set_title("Radiation vs Humidty")
sns.barplot(x="Humidity", y="Radiation",
            data=data, palette="GnBu_d", ax=ax3)
ax4.set_title("Radiation vs Wind Direction")
sns.barplot(x="WindDirection", y="Radiation",
            data=data, palette="GnBu_d", ax=ax4)
ax5.set_title("Radiation vs Wind Speed")
sns.barplot(x="WindSpeed", y="Radiation",
            data=data, palette="GnBu_d", ax=ax5)
ax6.set_title("Radiation vs Month")
sns.barplot(x="Month", y="Radiation",
            data=data, palette="GnBu_d", ax=ax6)
ax7.set_title("Radiation vs Hour")
sns.barplot(x="Hour", y="Radiation",
            data=data, palette="GnBu_d", ax=ax7)
ax8.set_title("Radiation vs Day")
sns.barplot(x="Day", y="Radiation",
            data=data, palette="GnBu_d", ax=ax8)

# Adjust spacing between subplots
fig.tight_layout(w_pad=3.0, h_pad=5.0)

# Hide x-axis values
ax1.set_xticklabels([])
ax2.set_xticklabels([])
ax3.set_xticklabels([])
ax4.set_xticklabels([])
ax5.set_xticklabels([])
ax6.set_xticklabels([])
ax7.set_xticklabels([])
ax8.set_xticklabels([])

In [0]:
#@title Divide clean dataset into training set and test set

# Sort dataset in descending order from the latest to the earliest
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by=['Date', 'Time'], ascending=False)

# Percentage of dataset to be considered as test set
test_split = 0.05
data_test = data[:][0:round((len(data)*test_split))]
data_train = data[:][round((len(data)*test_split)):]
# Shuffle training set
data_train = data_train.reindex(np.random.permutation(data_train.index))

print('Length of training dataset:', str(len(data_train)),
      '\nLength of test set length:', str(len(data_test)))

In [0]:
#@title Represent features as floating-point values

feature_columns = []

temperature = tf.feature_column.numeric_column("Temperature")
feature_columns.append(temperature)

pressure = tf.feature_column.numeric_column("Pressure")
feature_columns.append(pressure)

humidity = tf.feature_column.numeric_column("Humidity")
feature_columns.append(humidity)

wdirection = tf.feature_column.numeric_column("WindDirection")
feature_columns.append(wdirection)

wspeed = tf.feature_column.numeric_column("WindSpeed")
feature_columns.append(wspeed)

#sunrise = tf.feature_column.numeric_column("Sunrise")
#feature_columns.append(sunrise)

#sunset = tf.feature_column.numeric_column("Sunset")
#feature_columns.append(sunset)

#month = tf.feature_column.numeric_column("Month")
#feature_columns.append(month)

#day = tf.feature_column.numeric_column("Day")
#feature_columns.append(day)

hour = tf.feature_column.numeric_column("Hour")
feature_columns.append(hour)

#minute = tf.feature_column.numeric_column("Minute")
#feature_columns.append(minute)

# Convert the list of feature columns into a layer that 
# later will be fed into the model. 
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

print('Feature representations selected')

##Model: Linear Regression

In [0]:
#@title Define plotting function

"""Plot loss vs epoch in a graph"""
def plot_the_loss_curve(epochs, rmse_training, rmse_validation):
  
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Root Mean Squared Error')

  plt.plot(epochs[1:], rmse_training[1:], label='Training Loss')
  plt.plot(epochs[1:], rmse_validation[1:], label='Validation Loss')
  plt.legend()

  merged_rmse_lists = rmse_training[1:] + rmse_validation[1:]
  highest_loss = max(merged_rmse_lists)
  lowest_loss = min(merged_rmse_lists)
  delta = highest_loss - lowest_loss
  print(delta)

  top_of_y_axis = highest_loss + (delta * 0.05)
  bottom_of_y_axis = lowest_loss - (delta * 0.05)
  plt.ylim([bottom_of_y_axis, top_of_y_axis])
  plt.show()  

print('Plot function defined')

In [0]:
#@title Define functions to create and train  model

"""Create and compile a simple linear regression model."""
def create_model(learning_rate, feature_layer):
  
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Add the layer containing the feature columns to the model.
  model.add(feature_layer)

  # Add one linear layer to the model to yield a simple linear regressor.
  model.add(tf.keras.layers.Dense(units=1, input_shape=(1,)))

  # Construct the layers into a model that TensorFlow can execute.
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate),
                loss='mean_squared_error',
                metrics=[tf.keras.metrics.RootMeanSquaredError()])
  
  return model           

"""Feed a dataset into the model in order to train it."""
def train_model(model, dataset, epochs, batch_size,
                label_name, validation_split):

  # Split the dataset into features and label.
  features = {name:np.array(value) for name, value in dataset.items()}
  label = np.array(features.pop(label_name))
  history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, validation_split=validation_split, 
                      shuffle=True)

  # Get details that will be useful for plotting the loss curve.
  epochs = history.epoch
  hist = pd.DataFrame(history.history)
  rmse = hist['root_mean_squared_error']

  return epochs, rmse, history.history   

print('Model defined')

In [0]:
#@title Train model

# The following variables are the hyperparameters.
learning_rate = 0.01
epochs = 100
batch_size = 1024
validation_split = 0.5
label_name = 'Radiation'
 
# Establish the model's topography.
model = create_model(learning_rate, feature_layer)

# Train the model on the normalized training set.
epochs, rmse, history = train_model(model, data_train_norm, epochs, batch_size,
                                    label_name, validation_split=validation_split)
plot_the_loss_curve(epochs, history['root_mean_squared_error'], 
                    history['val_root_mean_squared_error'])

test_features = {name:np.array(value) for name, value in data_test_norm.items()}
# Isolate the label
test_label = np.array(test_features.pop(label_name))
print('\n Evaluate the linear regression model against the test set:')
model.evaluate(x=test_features, y=test_label, batch_size=batch_size)